In [3]:
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
 
# bu fonksiyon dizindeki her fotoğrafın özelliklerini çıkartır.
def extract_features(directory):
    #VGG 16 modelinden nesne olusturuldu
    model = VGG16()#onceden egitilmiş model olan imegent in agırılıklarından olusturulmus model cagırılır
    # model yeniden yapılandırıldı
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    # vgg 16 mimarı özeti:
    print(model.summary())
    features = dict()
    for name in listdir(directory):
        # dosyadan resim adları alındı
        filename = directory + '/' + name
        #goruntu yuklendi ve boyyutları 224,224,3 olarak düzenlendi
        image = load_img(filename, target_size=(224, 224,3))
        # Goruntuleri matris dizisine cevrildi.(ön işleme için)
        image = img_to_array(image)
        #görüntü boyutunu (batch_Size,height,width, channel) 'den (height,width, channel) yani (224,224,3)'ten (1,224,224,3) dönüştürür.
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        # goruntu tahmine hazır hale getirildi
        image = preprocess_input(image)
        # giriş örnekleri için çıktı tahminleri üretildi
        feature = model.predict(image)
        # foto id si
        image_id = name.split('.')[0]
        features[image_id] = feature
    return features
 


In [ ]:
directory = 'Flickr8k_Dataset'
features = extract_features(directory)
print('Su kadar özellik çıkarıldı: %d' % len(features))
dump(features, open('VGG16.pkl', 'wb'))

### METIN

In [2]:
import string
def yukle(dosya_ismi):
    file = open(dosya_ismi, 'r')
    text = file.read()
    file.close()
    return text
def aciklama_yukle(x):
    sozluk = dict()  
    for sira in x.split('\n'):
        satir = sira.split()
        if len(sira) < 2:
            continue
        resim_id, resim_aciklama = satir[0], satir[1:]
       
        resim_id = resim_id.split('.')[0]
        
        resim_aciklama = ' '.join(resim_aciklama)
       
        if resim_id not in sozluk:
            sozluk[resim_id] = list()
    
        sozluk[resim_id].append(resim_aciklama)
    return sozluk


def temizle(aciklamalar):
    y = str.maketrans('', '', string.punctuation)#punctuation noktalama isaretleridir.maketrans da noktalama isaretlerini bosluk ile değiştirmeye yarar
    for key, value in aciklamalar.items():
        uzunluk=len(value)
        for i in range(uzunluk):
            x = value[i]
            x = x.split()
            x = [word.lower() for word in x]
           # x = [w.translate(y) for w in x]
            x = [word for word in x if len(word)>1]
            value[i] =  ' '.join(x)
            
def dosyaya_kaydet(aciklamalar, dosya_adi):
    satir = list()
    for key, value in aciklamalar.items():
        for x in value:
            satir.append(key + ' ' + x)
    data = '\n'.join(satir)
    file = open(dosya_adi, 'w')
    file.write(data)
    file.close() 
    
    
    
dosya_adi= 'Flickr8k.token.txt'
#altyazıları yukle
x = yukle(dosya_adi)
#acıklamaları yukle
aciklamalar = aciklama_yukle(x)
print('yuklendı: %d ' % len(aciklamalar))
# kelimeleri temizle
temizle(aciklamalar)
# dosyaya kaydet
dosyaya_kaydet(aciklamalar,'altyazilar.txt')

yuklendı: 8092 


### DERİN ÖĞRENME MODEL TANIMI

In [1]:
from pickle import load
from array import array
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras.utils.vis_utils import model_to_dot
from tensorflow.python.keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint
 
def yukle(dosya_ismi):
    file = open(dosya_ismi, 'r')
    text = file.read()
    file.close()
    return text
#dataset resim acıklamalrı listesidir .jpg kaldırılmıs hali
def yukle_tamami(dosyaadi):
    x = yukle(dosyaadi)
    liste = list()
    for satir in x.split('\n'):
        if len(satir) < 1:
            continue
        y = satir.split('.')[0]
        liste.append(y)
    return set(liste)
 
# temizlenmiş açklamalar sözlüğü yükle
def temizle(aciklamalar):
    y = str.maketrans('', '', string.punctuation)#punctuation noktalama isaretleridir.maketrans da noktalama isaretlerini bosluk ile değiştirmeye yarar
    for key, value in aciklamalar.items():
        uzunluk=len(value)
        for i in range(uzunluk):
            x = value[i]
            x = x.split()
            x = [word.lower() for word in x]
            x = [w.translate(y) for w in x]
            x = [word for word in x if len(word)>1]
            value[i] =  'start'+' '.join(x)+'end'

# foto özelliklero yukle
def foto_ozellikle_yukle(dosyaadi, veriseti):# filename vgg16 dan cıkartılan özellik, train jpegsiz id
    tum_ozellikler = load(open(dosyaadi, 'rb'))
    ozellikler = {k: tum_ozellikler[k] for k in veriseti}
    return ozellikler
 
def id_al(aciklamalar):
    id_llistesi = list()
    for key in aciklamalar.keys():
        [id_llistesi.append(d) for d in aciklamalar[key]]
    return id_llistesi # burada teizleniş açıklama ve id sözcüğünden sadece key(id) alır
 
#temizlenmiş açıklamalar buraya gelir
def benersiz_tam_Sayi_olustur(aciklamalar):
    satirlar_id = id_al(aciklamalar) #train id
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(satirlar_id)# her train id için benzersiz farklı tam sayı verir
    return tokenizer
  
    
    
# en fazla kelime ile açıklamanın uzunluğunu hesaplama
def en_uzun_kelime(aciklamalar):
    satir = id_al(aciklamalar)
    return max(len(s.split()) for s in satir)
 
    
    
# bir görüntü için görüntü dizileri, giriş dizileri ve çıktı sözcükleri oluşturma
def dizi_olustur(tokenizer, max_kelime, aciklamalar, fotograf, kelime_boyutu):
    X1, X2, y = list(), list(), list()
    for key, value in aciklamalar.items():
        for x in value:
            # sırayı kodla
            s = tokenizer.texts_to_sequences([x])[0]  #sayısal ifadelerle belirtilir
            for i in range(1, len(s)):
                in_s, out_s = s[:i], s[i]
                in_s = pad_sequences([in_s], maxlen=max_kelime)[0] #giriş verilerinin sabit uzunlukta olmasını sağlar
                out_s = to_categorical([out_s], num_classes=kelime_boyutu)[0]#sınıf vektörünü kelime_boyutu kadar sınıf matrisine dönüştürür
                X1.append(fotograf[key][0])
                X2.append(in_s)
                y.append(out_s)
    return np.array(X1), np.array(X2), np.array(y)
 

# altyazı modeli tanımlama
def model_olusturma(sozluk_boyutu, max_kelime):
    # özellik çıkarıcı model(Kodlayıcı)
    inputs1 = Input(shape=(4096,))# model fotonun 4096 ogeden olusan vektor olmasını bekler ve fotografın 256 elemanlı temsilini üretmek için işler
    fe1 = Dropout(0.3)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)
     #Sequence Model
    inputs2 = Input(shape=(max_kelime,))
    se1 = Embedding(sozluk_boyutu, 256)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)
    # kod çözücü model
    decoder1 = add([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(sozluk_boyutu, activation='softmax')(decoder2)
    # [resm, sıra] [kelime] olacak sekilde bagla
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    print(model.summary())
    plot_model(model, to_file='model.png', show_shapes=True)
    return model
 
# eğitim veri setini yükle
dosyaadi = 'Flickr_8k.trainImages.txt'
egitim = yukle_tamami(dosyaadi)
print('Veri Seti(Eğitim): %d' % len(egitim))
# acıklamalar
egitim_aciklamalari = temizle('altyazilar.txt', egitim)# temizlenmiş açıklamalr sözlüğü
print('Açıklamalar: Eğitim=%d' % len(egitim_aciklamalari))
# özellikler
egitim_ozellikleri = foto_ozellikle_yukle('VGG16.pkl', train) # fotogtaf özzellikleri
print('Fotograf(Özellik): eğitim=%d' % len(egitim_ozellikleri))

benzersiz_Sayi = benersiz_tam_Sayi_olustur(egitim_aciklamalari) # temizlenmiş açıklamalar
sozcuk_boyutu = len(benzersiz_Sayi.word_index) + 1 #faklı kelime
print('Sözcük Boyutu: %d' % sozcuk_boyutu)
# max dizi uzunlugu belirleme
max_kelime = en_uzun_kelime(egitim_aciklamalari) #max acıklama uzunlugu
print('Açıklama uzunlugu: %d' % max_kelime)



X1train, X2train, ytrain = dizi_olustur(benzersiz_Sayi, max_kelime, egitim_aciklamalari, egitim_ozellikleri, sozcuk_boyutu)
 

dosyaadi = 'Flickr_8k.devImages.txt'
test = yukle_tamami(dosyaadi)
print('Veri Seti: Test= %d' % len(test))
# Acıklamalar
test_aciklamalari = temizle('altyazilar.txt', test)
print('Açıklama: Test=%d' % len(test_aciklamalari))
# Fotoğraf Özellikleri
test_ozellikleri = foto_ozellikle_yukle('VGG16.pkl', test)
print('Fotoğraf: Test=%d' % len(test_ozellikleri))


X1test, X2test, ytest = dizi_olustur(benzersiz_Sayi, max_kelime, test_aciklamalari, test_ozellikleri, sozcuk_boyutu)

model = model_olusturma(sozcuk_boyutu, max_kelime)

filepath = 'model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

model.fit([X1train, X2train], ytrain, epochs=20, verbose=2, callbacks=[checkpoint], validation_data=([X1test, X2test], ytest))


Veri Seti(Eğitim): 6000
Açıklamalar: Eğitim=6000
Fotograf(Özellik): eğitim=6000
Sözcük Boyutu: 7579
Açıklama uzunlugu: 34
Veri Seti: Test= 1000
Açıklama: Test=1000
Fotoğraf: Test=1000
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 34)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 4096)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 34, 256)      1940224     input_2[0][0]                    
__________________________________________________________________________________________________
dropout (

c:\users\lenovo\appdata\local\programs\python\python39\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/20
9576/9576 - 4656s - loss: 3.8078 - val_loss: 3.8835

Epoch 00002: val_loss improved from 4.03840 to 3.88346, saving model to model-ep002-loss3.808-val_loss3.883.h5
Epoch 3/20
9576/9576 - 4629s - loss: 3.5894 - val_loss: 3.8634

Epoch 00003: val_loss improved from 3.88346 to 3.86339, saving model to model-ep003-loss3.589-val_loss3.863.h5
Epoch 4/20
9576/9576 - 4839s - loss: 3.4777 - val_loss: 3.8575

Epoch 00004: val_loss improved from 3.86339 to 3.85746, saving model to model-ep004-loss3.478-val_loss3.857.h5
Epoch 5/20
9576/9576 - 4680s - loss: 3.4166 - val_loss: 3.8915

Epoch 00005: val_loss did not improve from 3.85746
Epoch 6/20
9576/9576 - 4572s - loss: 3.3778 - val_loss: 3.9342

Epoch 00006: val_loss did not improve from 3.85746
Epoch 7/20
9576/9576 - 4592s - loss: 3.3530 - val_loss: 3.9155

Epoch 00007: val_loss did not improve from 3.85746
Epoch 8/20
9576/9576 - 4585s - loss: 3.3372 - val_loss: 3.9610

Epoch 00008: val_loss did not improve from 3.85746
Epoch 9/20
9576

### DEGERLENDİRME

In [1]:
from numpy import argmax
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

def yukle(dosya_ismi):
    file = open(dosya_ismi, 'r')
    text = file.read()
    file.close()
    return text
#dataset resim acıklamalrı listesidir .jpg kaldırılmıs hali
def yukle_tamami(dosyaadi):
    x = yukle(dosyaadi)
    liste = list()
    for satir in x.split('\n'):
        if len(satir) < 1:
            continue
        y = satir.split('.')[0]
        liste.append(y)
    return set(liste)
 
# temizlenmiş açklamalar sözlüğü yükle
def temizle(aciklamalar):
    y = str.maketrans('', '', string.punctuation)#punctuation noktalama isaretleridir.maketrans da noktalama isaretlerini bosluk ile değiştirmeye yarar
    for key, value in aciklamalar.items():
        uzunluk=len(value)
        for i in range(uzunluk):
            x = value[i]
            x = x.split()
            x = [word.lower() for word in x]
            x = [w.translate(y) for w in x]
            x = [word for word in x if len(word)>1]
            value[i] =  'start'+' '.join(x)+'end'

# foto özelliklero yukle
def foto_ozellikle_yukle(dosyaadi, veriseti):# filename vgg16 dan cıkartılan özellik, train jpegsiz id
    tum_ozellikler = load(open(dosyaadi, 'rb'))
    ozellikler = {k: tum_ozellikler[k] for k in veriseti}
    return ozellikler
 
def id_al(aciklamalar):
    id_llistesi = list()
    for key in aciklamalar.keys():
        [id_llistesi.append(d) for d in aciklamalar[key]]
    return id_llistesi # burada teizleniş açıklama ve id sözcüğünden sadece key(id) alır
 
#temizlenmiş açıklamalar buraya gelir
def benersiz_tam_Sayi_olustur(aciklamalar):
    satirlar_id = id_al(aciklamalar) #train id
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(satirlar_id)# her train id için benzersiz farklı tam sayı verir
    return tokenizer
  
    
    
# en fazla kelime ile açıklamanın uzunluğunu hesaplama
def en_uzun_kelime(aciklamalar):
    satir = id_al(aciklamalar)
    return max(len(s.split()) for s in satir)
 
# bir tamsayıyı bir kelimeye eşleştirme
def tamsayieslestirme(integer, tokenizer):
    for kelime, index in tokenizer.word_index.items():
        if index == integer:
            return kelime
    return None
 
#bir resim için bir açıklama oluşturmak
def aciklama_olustur(model, tokenizer, foto, max_uzunluk):
    baslangic = 'startseq'
    #dizinin tüm uzunluğu boyunca yinelemek
    for i in range(max_uzunluk):
        # tamsayı kodlama giriş sırası
        s = tokenizer.texts_to_sequences([baslangic])[0]
        s = pad_sequences([s], maxlen=max_uzunluk)
        # sonraki kelimeyi tahmin etme
        y = model.predict([foto,s], verbose=0)
        # olasılığı tam sayıya çevirmek
        y = argmax(y)
        # tamsayıyı kelimeye eşleme
        kelime = tamsayieslestirme(y, tokenizer)
        # kelimeyi eşleyemezsek durdur
        if kelime is None:
            break
        #sonraki kelimeyi oluşturmak için girdi olarak ekle
        baslangic += ' ' + kelime
        #dizinin sonunu tahmin ederse durdue
        if kelime == 'endseq':
            break
    return baslangic
 
# modelin becerisini değerlendirme(BLEU SCORE)
def model_olustur(model, aciklamalar, fotograf, tokenizer, max_uzunluk):
    print("3")
    gercek, tahmin_edilen = list(), list()
    #tüm veri setinin üzerinden geç
    for key, value in aciklamalar.items():
        # acıklama üret
        y = aciklama_olustur(model, tokenizer, fotograf[key], max_uzunluk)
        #gercek deger ve tahmini değeri tut
        x = [v.split() for v in value]
        gercek.append(x)
        tahmin_edilen.append(y.split())
    # BLEU skoru hesapla
    print('BLEU-1: %f' % corpus_bleu(gercek, tahmin_edilen, weights=(1.0, 0, 0, 0)))
    print("BLEU 1 HESAPLANDI")
    #print('BLEU-2: %f' % corpus_bleu(gercek, tahmin_edilen, weights=(0.5, 0.5, 0, 0)))
    #print("BLEU 2 HESAPLANDI")
    #print('BLEU-3: %f' % corpus_bleu(gercek, tahmin_edilen, weights=(0.3, 0.3, 0.3, 0)))
    #print("BLEU 3 HESAPLANDI")
    #print('BLEU-4: %f' % corpus_bleu(gercek, tahmin_edilen, weights=(0.25, 0.25, 0.25, 0.25)))
    #print("BLEU 4 HESAPLANDI")


dosyaadi = 'Flickr_8k.trainImages.txt'
egitim = yukle_tamami(dosyaadi)
print('Veri Seti(Eğitim): %d' % len(egitim))
egitim_aciklamalari = temizle('altyazilar.txt', egitim)
print('Açıklama(Eğitim:%d' % len(egitim_aciklamalari))
tokenizer = benersiz_tam_Sayi_olustur(egitim_aciklamalari)
sozcuk_boyutu = len(tokenizer.word_index) + 1
print('Sözcuk Boyutu: %d' % sozcuk_boyutu)

max_kelime = en_uzun_kelime(egitim_aciklamalari)
print('Açıklama Uzunlugu: %d' % max_kelime)
 

dosyaadi = 'Flickr_8k.testImages.txt'
test = yukle_tamami(dosyaadi)
print('Veri Seti(Test): %d' % len(test))

test_aciklamalari = temizle('altyazilar.txt', test)
print('Açıklama(Test):%d' % len(test_aciklamalari))

test_ozellikleri = foto_ozellikle_yukle('VGG16.pkl', test)
print('Fotograf(Test):%d' % len(test_ozellikleri))
 

filename = 'epoch20_model-ep004-loss3.478-val_loss3.857.h5'
model = load_model(filename)
print("1")
model_olustur(model, test_aciklamalari, test_ozellikleri, tokenizer, max_kelime)
print("2")

Veri Seti(Eğitim): 6000


TypeError: temizle() takes 1 positional argument but 2 were given

### YENI ALTYAZI OLUSTURMA

In [ ]:
from pickle import load
from numpy import argmax
from keras.preprocessing.sequence import pad_sequences
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.models import load_model
 
def ozellik_cikar(dosyaadi):
    model = VGG16()
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    image = load_img(dosyaadi, target_size=(224, 224))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    feature = model.predict(image, verbose=0)
    return feature
 
# bir tamsayıyı bir kelimeye eşleştirme
def tamsayieslestirme(integer, tokenizer):
    for kelime, index in tokenizer.word_index.items():
        if index == integer:
            return kelime
    return None
 
#bir resim için bir açıklama oluşturmak
def aciklama_olustur(model, tokenizer, foto, max_uzunluk):
    baslangic = 'startseq'
    #dizinin tüm uzunluğu boyunca yinelemek
    for i in range(max_uzunluk):
        # tamsayı kodlama giriş sırası
        s = tokenizer.texts_to_sequences([baslangic])[0]
        s = pad_sequences([s], maxlen=max_uzunluk)
        # sonraki kelimeyi tahmin etme
        y = model.predict([foto,s], verbose=0)
        # olasılığı tam sayıya çevirmek
        y = argmax(y)
        # tamsayıyı kelimeye eşleme
        kelime = tamsayieslestirme(y, tokenizer)
        # kelimeyi eşleyemezsek durdur
        if kelime is None:
            break
        #sonraki kelimeyi oluşturmak için girdi olarak ekle
        baslangic += ' ' + kelime
        #dizinin sonunu tahmin ederse durdue
        if kelime == 'endseq':
            break
    return baslangic

tokenizer = load(open('tokenizer.pkl', 'rb'))
# maksimum dizi uzunluğunu önceden tanımlandı
max_uzunluk = 34

model = load_model('epoch20_model-ep004-loss3.478-val_loss3.857.h5')

fotograf = ozellik_cikar('aaa.jpg')
# açıklama oluştura
aciklama = aciklama_olustur(model, tokenizer, fotograf, max_uzunluk)
print(aciklama)
#man is surfing in the ocea